In [ ]:
# SETUP ENVIRONMENT
!pip install -q langchain faiss-cpu transformers accelerate bitsandbytes huggingface_hub sentence-transformers langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install assemblyai gtts

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken Ngrok_Authtoken

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

# Set your Hugging Face token for authentication
os.environ["HUGGINGFACE_TOKEN"] = "Huggingface Token"

# Set a local cache directory for transformers to speed up model loading
os.environ["TRANSFORMERS_CACHE"] = "/content/hf_cache"
Path("/content/hf_cache").mkdir(exist_ok=True)

In [ ]:
# AssemblyAI token
aai.settings.api_key = "Assembly_Token"

In [ ]:
# Imports for RAG system, embeddings, and LLM
import re
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# System utilities
import shutil
import io
from base64 import b64decode

# Audio / Speech processing
import assemblyai as aai
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output

In [ ]:
# Paths and directories
VECTORSTORE_DIR = Path("./vectorstores_local")
VECTORSTORE_DIR.mkdir(exist_ok=True)

# Model configuration
EMBEDDING_MODEL = "intfloat/multilingual-e5-large"
LLM_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
FALLBACK_MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Excel files containing datasets
PROJECTS_XLSX = "/content/English_Projects_Cleaned.xlsx"
CITIES_XLSX = "/content/English_Cities_Cleaned.xlsx"
PEOPLE_XLSX = "/content/English_Characters_Cleaned.xlsx"

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
def load_excel_safe(path: str):
    if not Path(path).exists():
        print(f"File not found: {path}")
        return None

    df = pd.read_excel(path)
    cols = [c.lower() for c in df.columns]

    # Ensure 'title' column exists
    if "title" not in cols:
        df.rename(columns={df.columns[0]: "title"}, inplace=True)

    # Ensure 'content' column exists
    if "content" not in cols:
        if len(df.columns) > 1:
            df.rename(columns={df.columns[1]: "content"}, inplace=True)
        else:
            df["content"] = df["title"]  # Fallback if only one column

    return df

# Load datasets safely
datasets = {
    "people": load_excel_safe(PEOPLE_XLSX),
    "projects": load_excel_safe(PROJECTS_XLSX),
    "Cities": load_excel_safe(CITIES_XLSX),
}

# Print summary of loaded datasets
for k, v in datasets.items():
    if v is not None:
        print(f"[Loaded] {k}: {v.shape}")

[✅ Loaded] people: (101, 3)
[✅ Loaded] projects: (47, 3)
[✅ Loaded] Cities: (189, 6)


In [ ]:
VECTORSTORE_DIR = Path("./vectorstores_local")

# Delete existing FAISS vectorstores to start fresh
if VECTORSTORE_DIR.exists():
    shutil.rmtree(VECTORSTORE_DIR)
    print("🧹 Deleted old FAISS vectorstores.")

# Recreate the directory for storing vectorstores
VECTORSTORE_DIR.mkdir(exist_ok=True)

🧹 Deleted old FAISS vectorstores.


In [ ]:
def df_to_docs(df, category):
    # Define chunk sizes and overlap for each category
    chunk_cfg = {
        "people": (700, 120),
        "Cities": (300, 50),
        "projects": (500, 80),
    }
    size, overlap = chunk_cfg.get(category, (500, 80))

    # Initialize the text splitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)

    docs = []
    # Iterate through DataFrame rows and split content into chunks
    for _, r in df.iterrows():
        title, content = str(r["title"]), str(r["content"])
        for c in splitter.split_text(content):
            docs.append(Document(page_content=c, metadata={"title": title, "category": category}))

    return docs

# Convert all loaded datasets into documents
docs_by_cat = {cat: df_to_docs(df, cat) for cat, df in datasets.items() if df is not None}

In [ ]:
# Initialize the embeddings model for all categories
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

def build_or_load_vs(cat, docs):
    path = VECTORSTORE_DIR / cat
    if path.exists():
        # Load existing vectorstore if available
        print(f"[Load FAISS] {cat}")
        vs = FAISS.load_local(str(path), embeddings, allow_dangerous_deserialization=True)
    else:
        # Build new vectorstore from documents and save it locally
        print(f"[Build FAISS] {cat} ({len(docs)} docs)")
        vs = FAISS.from_documents(docs, embeddings)
        vs.save_local(str(path))
        print(f"[Saved] {path}")
    return vs

# Build or load FAISS vectorstores for all categories
vectorstores = {cat: build_or_load_vs(cat, docs) for cat, docs in docs_by_cat.items()}

/tmp/ipython-input-1367479060.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

[⚙️ Build FAISS] people (1010 docs)
[💾 Saved] vectorstores_local/people
[⚙️ Build FAISS] projects (174 docs)
[💾 Saved] vectorstores_local/projects
[⚙️ Build FAISS] Cities (189 docs)
[💾 Saved] vectorstores_local/Cities


In [ ]:
def load_local_llm(model_id=LLM_MODEL_ID, fallback_id=FALLBACK_MODEL_ID):
    try:
        print(f"Loading LLM: {model_id}")
        # Load tokenizer for the primary model
        tok = AutoTokenizer.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"])
        # Load model with 4-bit precision to save memory
        mdl = AutoModelForCausalLM.from_pretrained(
            model_id,
            token=os.environ["HUGGINGFACE_TOKEN"],
            device_map="auto",
            torch_dtype="auto",
            load_in_4bit=True
        )
    except Exception as e:
        # Fallback to smaller model if loading fails
        print(f"[Fallback to {fallback_id}] Reason:", e)
        tok = AutoTokenizer.from_pretrained(fallback_id)
        mdl = AutoModelForCausalLM.from_pretrained(fallback_id, device_map="auto", torch_dtype="auto")

    # Wrap the model in a HuggingFace pipeline for text generation
    gen = pipeline("text-generation", model=mdl, tokenizer=tok, max_new_tokens=256, temperature=0.2)
    return HuggingFacePipeline(pipeline=gen)

# Load the LLM
llm = load_local_llm()

🚀 Loading LLM: mistralai/Mistral-7B-Instruct-v0.2


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-3788843936.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=gen)


In [ ]:
# Create a prompt template for the RAG system
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Answer the following question based ONLY on the provided context.\n"
        "If the context does not contain the answer, say 'Not enough information.'\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n\n"
        "Answer clearly and concisely:"
    ),
)

# Initialize RAG systems for each category
rag_systems = {}
for cat, vs in vectorstores.items():
    # Convert FAISS vectorstore into a retriever
    retriever = vs.as_retriever(search_kwargs={"k": 4})

    # Build RetrievalQA chain using the LLM and the retriever
    rag = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",  # Concatenate retrieved docs as input
        chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "context"},
    )

    # Store the RAG system for this category
    rag_systems[cat] = rag
    print(f"[RAG ready] {cat}")

[🤖 RAG ready] people
[🤖 RAG ready] projects
[🤖 RAG ready] Cities


In [ ]:
# Define metadata descriptions for each category
CATEGORY_METADATA = {
    "people": "Biographies and achievements of Saudi figures like King Abdulaziz, King Salman, Crown Prince Mohammed bin Salman, and others.",
    "projects": "Vision 2030 projects such as NEOM, Qiddiya, ROSHN, Riyadh Art, The Line.",
    "Cities": "Information about Saudi cities and landmarks such as Riyadh, Jeddah, Abha, AlUla, Taif."
}

# Precompute embeddings for the category metadata for semantic routing
category_embeddings = {k: embeddings.embed_query(v) for k, v in CATEGORY_METADATA.items()}

# Function to route a user query to the most relevant category
def route_query_semantic_local(query: str, threshold=0.18):
    # Embed the query
    q_vec = embeddings.embed_query(query)
    # Compute cosine similarity with each category
    sims = {c: float(cosine_similarity([q_vec], [v])[0][0]) for c, v in category_embeddings.items()}
    print("Similarities:", sims)
    # Pick the category with the highest similarity
    best = max(sims, key=sims.get)
    # Fallback rules if similarity is below threshold
    if sims[best] < threshold:
        ql = query.lower()
        if any(x in ql for x in ["who", "king", "prince", "mohammed", "bin salman"]): return "people"
        if any(x in ql for x in ["project", "vision", "initiative"]): return "projects"
        if any(x in ql for x in ["city", "visit", "where", "travel"]): return "Cities"
    return best

# Function to clean the raw RAG output by removing instruction echoes
def clean_echo(raw: str):
    t = re.sub(r"(?s).*Answer (clearly|concisely|directly).*?:", "", raw).strip()
    return re.sub(r"\s+", " ", t).strip()

In [ ]:
# Function for interactive RAG Q&A loop in console
def ask_loop():
    print("Smart local RAG ready. Type 'exit' to quit.")
    while True:
        # Take user input
        q = input("\nAsk: ").strip()
        if q.lower() in ("exit", "quit"):
            break

        cat = route_query_semantic_local(q)
        print(f"Routed to: {cat}")

        # Retrieve the RAG system for the chosen category
        rag = rag_systems.get(cat)
        if not rag:
            print("No RAG for this category.")
            continue

        retriever = rag.retriever
        docs = retriever.invoke(q)
        top_docs = docs[:3]

        context = "\n\n".join([
            f"Source {i+1}:\n{d.page_content}" for i, d in enumerate(top_docs)
        ])
        print(f"Used {len(top_docs)} chunks for context.")

        # Generate the answer using RAG system
        raw = rag.invoke({"context": context, "query": q})
        answer = clean_echo(raw["result"] if isinstance(raw, dict) and "result" in raw else str(raw))

        print(f"\nQuestion: {q}")
        print(f"Answer: {answer}")
        print("=" * 60)

In [ ]:
ask_loop()

✅ Smart local RAG ready. Type 'exit' to quit.

Ask: How does Saudi Vision 2030 contribute to tourism development?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📊 Similarities: {'people': 0.7894015631024627, 'projects': 0.8372988332673157, 'Cities': 0.790574336041463}
🧭 Routed to: projects
🧩 Used 3 chunks for context.

💡 Question: How does Saudi Vision 2030 contribute to tourism development?
💬 Answer: Saudi Vision 2030 contributes to tourism development by focusing on developing the tourism and entertainment sectors in the Kingdom, showcasing its natural treasures, and offering unparalleled experiences. Specifically, the context discusses the development of Shura Island, which preserves its biodiversity, enhances its natural environment, and features modern resort and hotel designs.

Ask: exit


In [ ]:
def record_audio_js():
    display(Javascript('''
    const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
    const record = time => new Promise(async resolve => {
      const stream = await navigator.mediaDevices.getUserMedia({ audio: true })
      const recorder = new MediaRecorder(stream)
      const data = []
      recorder.ondataavailable = event => data.push(event.data)
      recorder.start()
      await sleep(time)
      recorder.stop()
      await new Promise(resolve => recorder.onstop = resolve)
      const blob = new Blob(data, { type: 'audio/wav' })
      const arrayBuffer = await blob.arrayBuffer()
      const base64 = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)))
      google.colab.kernel.invokeFunction('notebook.get_recording', [base64], {})
    })
    record(5000)  // Start recording for 5 seconds
    '''))

# Global variable to store the recorded audio
recorded_audio = None

# Callback function to receive Base64 audio from JS
def get_recording(b64_audio):
    global recorded_audio
    recorded_audio = io.BytesIO(b64decode(b64_audio))
    print("Recording captured successfully!")

# Register the callback with Colab
output.register_callback('notebook.get_recording', get_recording)

In [ ]:
record_audio_js()

<IPython.core.display.Javascript object>

🎙️ Recording captured successfully!


In [ ]:
# Function to convert recorded audio to text using AssemblyAI
def speech_to_text():
    # Check if audio is available
    if recorded_audio is None:
        print("No recording detected.")
        return ""

    # Initialize AssemblyAI transcriber
    transcriber = aai.Transcriber()

    # Transcribe the recorded audio
    transcript = transcriber.transcribe(recorded_audio)

    # Extract the text, remove leading/trailing spaces
    text = transcript.text.strip() if transcript.text else ""

    print(f"You said: {text}")
    return text

# Example usage: convert the latest recorded audio to text
q = speech_to_text()

🗣️ You said: Where can I go in jeddah?


In [ ]:
# Function to get a RAG-generated answer for a given query
def rag_answer(q: str):
    # Route the query to the most relevant category
    cat = route_query_semantic_local(q)

    # Retrieve the RAG system for that category
    rag = rag_systems.get(cat)
    if not rag:
        return "Sorry, I don’t have data for that category."

    # Retrieve relevant documents
    retriever = rag.retriever
    docs = retriever.invoke(q)

    # Take the top 3 chunks for context
    top_docs = docs[:3]
    context = "\n\n".join([
        f"Source {i+1}:\n{d.page_content}" for i, d in enumerate(top_docs)
    ])

    # Generate the answer using the RAG system
    raw = rag.invoke({"context": context, "query": q})
    answer = clean_echo(raw["result"] if isinstance(raw, dict) and "result" in raw else str(raw))

    return answer

In [ ]:
# Function to get a RAG answer from a voice query
def rag_answer_voice(query):
    if not query:
        return

    # Route query to the most relevant category
    cat = route_query_semantic_local(query)
    print(f"Routed to: {cat}")

    # Get the RAG system for that category
    rag = rag_systems.get(cat)
    if not rag:
        print("No RAG for this category.")
        return

    # Retrieve relevant documents
    retriever = rag.retriever
    docs = retriever.invoke(query)

    # Use top 3 chunks as context
    context = "\n\n".join([
        f"Source {i+1}:\n{d.page_content}" for i, d in enumerate(docs[:3])
    ])

    # Generate answer using RAG
    raw = rag.invoke({"context": context, "query": query})
    answer = clean_echo(raw["result"] if isinstance(raw, dict) and "result" in raw else str(raw))

    # Print and return the answer
    print(f"{answer}")
    return answer

# Function to convert text to speech
def text_to_speech(text):
    tts = gTTS(text=text, lang="en")
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

In [ ]:
record_audio_js()

<IPython.core.display.Javascript object>

🎙️ Recording captured successfully!


In [ ]:
# Capture user speech and convert it to text
q = speech_to_text()

# Get RAG answer for the transcribed question
answer = rag_answer_voice(q)

# Convert the RAG answer to speech and play it
text_to_speech(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You said: Who is the king of Saudi Arabia?
📊 Similarities: {'people': 0.8329609534975975, 'projects': 0.7529228596279827, 'Cities': 0.8050122410591485}
🧭 Routed to: people
💬 King Salman Bin Abdulaziz Bin Abdulrahman Bin Faisal Bin Turki Bin Abdullah Bin Mohammed Bin Saud is the current king of Saudi Arabia.


In [ ]:
print(rag_answer("Tell me about NEOM"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📊 Similarities: {'people': 0.7583109141727007, 'projects': 0.8135794030792883, 'Cities': 0.7643217067028774}
NEOM is a futuristic city project in northwest Saudi Arabia, launched in 2017 by Crown Prince Mohammed bin Salman, aimed at transforming the desert into a smart, sustainable city powered entirely by renewable energy. The project includes major developments like The Line, Oxagon, Trojena, and Sindalah, and envisions a new model for sustainable living, work, and prosperity. Oxagon, a floating coastal city, connects residents and businesses to global markets and features an integrated port and logistics hub. Trojena will host global events and festivals, and NEOM represents Saudi Arabia's commitment to a sustainable, technologically advanced future.


In [ ]:
record_audio_js()

<IPython.core.display.Javascript object>

🎙️ Recording captured successfully!


In [ ]:
q = speech_to_text()
answer = rag_answer_voice(q)
text_to_speech(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You said: Where can I go in jeddah?
📊 Similarities: {'people': 0.7769664048176991, 'projects': 0.7906762612081156, 'Cities': 0.8167826588621483}
🧭 Routed to: Cities
💬 You can go for a walk in Jeddah, drive around Jeddah Circuit, enjoy the Jeddah Sea Breacher, or go scuba diving with Jeddah TEC Divers.


In [ ]:
# Function to convert audio bytes to text using AssemblyAI
def speech_to_text(audio_bytes):
    transcriber = aai.Transcriber()
    transcript = transcriber.transcribe(audio_bytes)
    text = transcript.text.strip() if transcript.text else ""
    print(f"You said: {text}")
    return text

In [ ]:
# Flask app for serving RAG as text & voice endpoints

from flask import Flask, request, jsonify, send_file
import io
import base64
from gtts import gTTS
from pyngrok import ngrok

app = Flask(__name__)


# Text endpoint
@app.route("/ask", methods=["POST"])
def ask_text():
    data = request.get_json()
    query = data.get("text", "")
    if not query:
        return jsonify({"answer": "No query provided."})
    answer = rag_answer(query)  # Call RAG model
    return jsonify({"answer": answer})


# Voice endpoint
@app.route("/voice", methods=["POST"])
def voice():
    audio_file = request.files.get("audio")
    if not audio_file:
        return jsonify({"error": "No audio uploaded"}), 400

    # Convert uploaded audio to bytes
    audio_bytes = io.BytesIO(audio_file.read())

    # Transcribe audio to text
    query_text = speech_to_text(audio_bytes)

    # Get RAG answer
    answer_text = rag_answer(query_text)

    # Convert answer text to speech (MP3)
    tts = gTTS(text=answer_text, lang="en")
    mp3_fp = io.BytesIO()
    tts.write_to_fp(mp3_fp)
    mp3_fp.seek(0)

    # Encode audio as Base64 for JSON response
    audio_base64 = base64.b64encode(mp3_fp.read()).decode("utf-8")

    # Return JSON with text and audio
    return jsonify({"answer": answer_text, "audio_data": audio_base64})


# Start Flask via ngrok tunnel
public_url = ngrok.connect(5000)
print("ngrok tunnel URL:", public_url)

app.run(port=5000)

🌐 ngrok tunnel URL: NgrokTunnel: "https://millicent-hypogynous-basally.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You said: Where can I go in jeddah?
📊 Similarities: {'people': 0.7769664048176991, 'projects': 0.7906762612081156, 'Cities': 0.8167826588621483}


INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 15:00:07] "POST /voice HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You said: Where can I go in jetta?
📊 Similarities: {'people': 0.7462140529408973, 'projects': 0.7495456485057991, 'Cities': 0.7726519372745291}


INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 15:04:06] "POST /voice HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You said: Where can I go in jeddah?
📊 Similarities: {'people': 0.7769664048176991, 'projects': 0.7906762612081156, 'Cities': 0.8167826588621483}


INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 15:04:26] "POST /voice HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📊 Similarities: {'people': 0.760458427354501, 'projects': 0.809688857437415, 'Cities': 0.7710861031187322}


INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 15:05:00] "POST /ask HTTP/1.1" 200 -
